In [10]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np
import os

In [2]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'D:\\data\\0643'

In [3]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

In [6]:
def get_premuted(x):
    premuted_x = []
    for i in range(len(x)):
        rnd_index = np.random.randint(0, len(x))
        new_x = x[:rnd_index+1]+x[rnd_index:]
        premuted_x.append(new_x)
    for item in [x + x[-1], x + x[-1]*2, x[:-1], x]:
        premuted_x.append(item)
        
    return premuted_x

In [11]:
for file in os.listdir(data_path):
    input_set = set()
    target_set = set()
    with open(os.path.join(data_path, file), "r", encoding="utf-8") as fp:
        for line in fp:
            line_split = line.split()
            if len(line_split) > 1:
                inp, target = line_split[0], line_split[1]
            else:
                continue
            
            if len(inp) > 0 and len(target) > 0:
                premuted_inp = get_premuted(inp.lower())
                
                for item in premuted_inp:
                    if len(item) > 1 and item not in input_set:
                        input_texts.append(item)
                        target_texts.append("\t" + target.lower() + "\n")
                        input_set.add(item)
                        
                
                        
                input_characters.update(inp.lower())
                target_characters.update(target.lower() + "\t\n")
                    
                
        

In [12]:
len(input_texts), len(target_texts)

(303726, 303726)

In [15]:
input_texts[20:30]

['beatefful',
 'beeateful',
 'beatefull',
 'beatefulll',
 'beatefu',
 'beateful',
 'divve',
 'divee',
 'ddive',
 'diveee']

In [16]:
target_texts[20:30]

['\tbeautiful\n',
 '\tbeautiful\n',
 '\tbeautiful\n',
 '\tbeautiful\n',
 '\tbeautiful\n',
 '\tbeautiful\n',
 '\tdrive,\n',
 '\tdrive,\n',
 '\tdrive,\n',
 '\tdrive,\n']

In [17]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [18]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 303726
Number of unique input tokens: 56
Number of unique output tokens: 60
Max sequence length for inputs: 27
Max sequence length for outputs: 28


In [19]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

In [20]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [21]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [22]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]


In [23]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


In [24]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)


In [25]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=5,
          validation_split=0.2)


Train on 242980 samples, validate on 60746 samples
Epoch 1/5
242980/242980 [==============================] - 293s 1ms/step - loss: 0.3139 - acc: 0.0965 - val_loss: 0.5201 - val_acc: 0.1682
Epoch 2/5
242980/242980 [==============================] - 288s 1ms/step - loss: 0.1667 - acc: 0.1396 - val_loss: 0.4645 - val_acc: 0.1903
Epoch 3/5
242980/242980 [==============================] - 288s 1ms/step - loss: 0.1172 - acc: 0.1535 - val_loss: 0.4791 - val_acc: 0.1944
Epoch 4/5
242980/242980 [==============================] - 296s 1ms/step - loss: 0.0920 - acc: 0.1603 - val_loss: 0.5141 - val_acc: 0.1940
Epoch 5/5
242980/242980 [==============================] - 288s 1ms/step - loss: 0.0768 - acc: 0.1645 - val_loss: 0.5486 - val_acc: 0.1931


In [135]:
# Save model
model.save('s2s.h5')

C:\Users\anshu\AppData\Local\Continuum\anaconda3\envs\mlenv\lib\site-packages\keras\engine\topology.py:2379: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


In [26]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [27]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [28]:
for seq_index in range(1100, 1150):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: consentrat
Decoded sentence: concentrated

-
Input sentence: consentrate
Decoded sentence: concentrated

-
Input sentence: dimenntion
Decoded sentence: dimension

-
Input sentence: ddimention
Decoded sentence: dimension

-
Input sentence: dimmention
Decoded sentence: dimension

-
Input sentence: dimentioon
Decoded sentence: dimension

-
Input sentence: dimentiion
Decoded sentence: dimensions

-
Input sentence: dimenttion
Decoded sentence: dimension

-
Input sentence: dimentionn
Decoded sentence: dimension

-
Input sentence: dimentionnn
Decoded sentence: dimension

-
Input sentence: dimentio
Decoded sentence: dimension

-
Input sentence: dimention
Decoded sentence: dimension

-
Input sentence: seingg
Decoded sentence: seeing

-
Input sentence: seeing
Decoded sentence: seeing

-
Input sentence: sseing
Decoded sentence: seeing

-
Input sentence: seinggg
Decoded sentence: seeing

-
Input sentence: sein
Decoded sentence: $seising

-
Input sentence: seing
Decoded sentence: 

In [74]:
encoder_input_test = np.zeros(
    (1, max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
text = 'rela'
for t, char in enumerate(text):
    encoder_input_test[0, t, input_token_index[char]] = 1.

decoded_sentence = decode_sequence(encoder_input_test)
print(text)
print(decoded_sentence)

rela
relax



In [37]:
input_token_index['o']

44

In [30]:
encoder_input_data[10: 10 + 1].shape

(1, 27, 56)